In [39]:
from neo4j import GraphDatabase
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
from statistics import harmonic_mean


pd.set_option('display.max_rows', 500)

In [40]:

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("test", "666666")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 

    driver.verify_connectivity() 

cell_type_now='astrocyte of the cerebral cortex'

query_jaccard="""
MATCH (ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g:Gene)
WITH g
MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup)
RETURN s1.species_scientific_name, count(DISTINCT og) as num_og"""


with driver.session() as session:
    result_ct = session.run(query_jaccard)
    df_ct = result_ct.to_df()

df_ct

query2="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g1:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
    (g1:Gene)-[:GeneFromSpecies]->(s1),
    (s2:Species)<-[:CellTypeFromSpecies]-(ct)<-[:GeneEnrichedInCellType]-(g2:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
    (g2:Gene)-[:GeneFromSpecies]->(s2)
WHERE s1 <> s2
WITH s1, s2, COUNT(DISTINCT g1) AS genes1, COUNT(DISTINCT g2) AS genes2, ct, count(DISTINCT gf) AS num_og
RETURN s1.species_scientific_name, s2.species_scientific_name, genes1, genes2, num_og"""

with driver.session() as session:
    result2 = session.run(query2)
    df2 = result2.to_df()


df2

In [41]:
query_ct = """
MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup {eggnog_dataset_name: 'primates'})<-[:GeneInOrthologousGroup]-(g2:Gene)-[:GeneFromSpecies]->(s2:Species),
(g)-[:GeneEnrichedInCellType]->(c:CellType)<-[:GeneEnrichedInCellType]-(g2)
WHERE id(s1) <> id(s2)
RETURN s1.species_scientific_name, s2.species_scientific_name, c.cell_type_name, size(collect(DISTINCT og)) AS intersection
ORDER BY s1.species_scientific_name, s2.species_scientific_name
"""


with driver.session() as session:
    result_ct = session.run(query_ct)
    df_ct = result_ct.to_df()

In [42]:
df_ct

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,intersection
0,cjacchus,ggorilla,vip GABAergic cortical interneuron,13
1,cjacchus,ggorilla,sncg GABAergic cortical interneuron,9
2,cjacchus,ggorilla,microglial cell,206
3,cjacchus,ggorilla,vascular leptomeningeal cell,203
4,cjacchus,ggorilla,oligodendrocyte precursor cell,63
...,...,...,...,...
415,ptroglodytes,mmulatta,caudal ganglionic eminence derived GABAergic c...,23
416,ptroglodytes,mmulatta,vip GABAergic cortical interneuron,29
417,ptroglodytes,mmulatta,sncg GABAergic cortical interneuron,23
418,ptroglodytes,mmulatta,sst GABAergic cortical interneuron,17


In [8]:
## this is for each cell type, want to do this for broad type at different levels

In [43]:
cell_type_col = 'cell_type_name'

In [44]:

# all-by-all cell tyope enriched genes belongs to the same OG
# note the change to counting genes, instead of OGs, to account for many in-paralogs

def get_cell_type_intersection(cell_type_col:str):

    query_ct_all = f"""
    MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup)<-[:GeneInOrthologousGroup]-(g2:Gene)-[:GeneFromSpecies]->(s2:Species),
    (g)-[:GeneEnrichedInCellType]->(c:CellType),
    (c2:CellType)<-[:GeneEnrichedInCellType]-(g2)
    WHERE id(s1) <> id(s2)
    RETURN s1.species_scientific_name, s2.species_scientific_name, c.{cell_type_col}, c2.{cell_type_col}, og.eggnog_dataset_name, size(collect(DISTINCT g)) AS num_gene_sp1, size(collect(DISTINCT g2)) AS num_gene_sp2
    ORDER BY s1.species_scientific_name, s2.species_scientific_name, c.{cell_type_col}, c2.{cell_type_col}, og.eggnog_dataset_name
    """

    ## all by all cell type enriched genes belongs to the same OG

    with driver.session() as session:
        result_ct_all = session.run(query_ct_all)
        df_ct_all = result_ct_all.to_df()

    return df_ct_all



In [45]:
get_cell_type_intersection('broad_type').to_csv("cell_type_enriched_genes_same_og_all_to_all_all_datasets_broad_type.csv", index=False)
get_cell_type_intersection('cell_type_name').to_csv("cell_type_enriched_genes_same_og_all_to_all_all_datasets_cell_type_name.csv", index=False)    
get_cell_type_intersection('broad_type_2').to_csv("cell_type_enriched_genes_same_og_all_to_all_all_datasets_broad_type_2.csv", index=False)    
get_cell_type_intersection('broad_type_3').to_csv("cell_type_enriched_genes_same_og_all_to_all_all_datasets_broad_type_3.csv", index=False)      

In [12]:
query_score= """MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup)<-[:GeneInOrthologousGroup]-(g2:Gene)-[:GeneFromSpecies]->(s2:Species),
(g)-[e1:GeneEnrichedInCellType]->(c:CellType)<-[e2:GeneEnrichedInCellType]-(g2)
WHERE id(s1) <> id(s2)
RETURN s1.species_scientific_name, s2.species_scientific_name, c.cell_type_name, og.id, g.external_gene_name, g2.external_gene_name, e1.specificity_score, e2.specificity_score
ORDER BY s1.species_scientific_name, s2.species_scientific_name"""

with driver.session() as session:
    result_score = session.run(query_score)
    df_score = result_score.to_df()

In [13]:
df_score

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,og.id,g.external_gene_name,g2.external_gene_name,e1.specificity_score,e2.specificity_score
0,cjacchus,ggorilla,astrocyte of the cerebral cortex,5017S,TTYH1,TTYH1,7.358435,10.983144
1,cjacchus,ggorilla,astrocyte of the cerebral cortex,9GUX8,TTYH1,TTYH1,7.358435,10.983144
2,cjacchus,ggorilla,astrocyte of the cerebral cortex,HUZ5P,TTYH1,TTYH1,7.358435,10.983144
3,cjacchus,ggorilla,astrocyte of the cerebral cortex,8ZN8V,TTYH1,TTYH1,7.358435,10.983144
4,cjacchus,ggorilla,astrocyte of the cerebral cortex,H4SNG,TTYH1,TTYH1,7.358435,10.983144
...,...,...,...,...,...,...,...,...
222459,ptroglodytes,mmulatta,oligodendrocyte,9GPC5,PARD3B,PARD3B,4.141009,12.479415
222460,ptroglodytes,mmulatta,oligodendrocyte,8Z0DV,PARD3B,PARD3B,4.141009,12.479415
222461,ptroglodytes,mmulatta,oligodendrocyte,HVEU5,PARD3B,PARD3B,4.141009,12.479415
222462,ptroglodytes,mmulatta,oligodendrocyte,H5A47,PARD3B,PARD3B,4.141009,12.479415


In [14]:
df_score.to_csv("intersection_enriched_score_metazoa.csv")

In [15]:

def get_enrichment_score(cell_type_col:str):


    query_score_all= f"""MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup)<-[:GeneInOrthologousGroup]-(g2:Gene)-[:GeneFromSpecies]->(s2:Species),
    (g)-[e1:GeneEnrichedInCellType]->(c:CellType),
    (c2:CellType)<-[e2:GeneEnrichedInCellType]-(g2)
    WHERE id(s1) <> id(s2)
    RETURN s1.species_scientific_name, s2.species_scientific_name, c.{cell_type_col}, c2.{cell_type_col}, og.id, og.eggnog_dataset_name, g.external_gene_name, g2.external_gene_name, e1.specificity_score, e2.specificity_score
    ORDER BY s1.species_scientific_name, s2.species_scientific_name, c.{cell_type_col}, c2.{cell_type_col}, og.eggnog_dataset_name"""


    ## all by all cell type enriched genes belongs to the same OG

    with driver.session() as session:
        result_score_all = session.run(query_score_all)
        df_score_all = result_score_all.to_df()
        df_score_all = df_score_all.drop_duplicates()

    return df_score_all

In [16]:
get_enrichment_score("cell_type_name").to_csv("intersection_enriched_score_all_datasets_cell_type_name.csv")
get_enrichment_score("broad_type").to_csv("intersection_enriched_score_all_datasets_broad_type.csv")
get_enrichment_score("broad_type_2").to_csv("intersection_enriched_score_all_datasets_broad_type_2.csv")
get_enrichment_score("broad_type_3").to_csv("intersection_enriched_score_all_datasets_broad_type_3.csv")

StopIteration: 

query_union = """MATCH (s1:Species)<-[:GeneFromSpecies]-(g1:Gene)-[:GeneInOrthologousGroup]->(og1:OrthologousGroup),
(s2:Species)<-[:GeneFromSpecies]-(g2:Gene)-[:GeneInOrthologousGroup]->(og2:OrthologousGroup),
(g1)-[:GeneEnrichedInCellType]->(c:CellType)<-[:GeneEnrichedInCellType]-(g2)
WHERE id(s1) <> id(s2)
RETURN s1.species_scientific_name, s2.species_scientific_name, c.cell_type_name, size(collect(DISTINCT og1)) AS og_1, size(collect(DISTINCT og2)) AS og_2
ORDER BY s1.species_scientific_name, s2.species_scientific_name
"""

with driver.session() as session:
    result_union = session.run(query_union)
    df_union = result_union.to_df()

df_union


In [56]:

# get all enriched genes in cell type, including orphan genes that does not belong to a OG, which is allowed

def get_all_enriched(cell_type_col:str):


    query_perct_all = f"""MATCH (s1:Species)<-[:GeneFromSpecies]-(g1:Gene),
    (g1)-[:GeneEnrichedInCellType]->(c:CellType)
    WITH s1, g1, c
    OPTIONAL MATCH (g1:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup)
    RETURN s1.species_scientific_name, c.{cell_type_col}, og.eggnog_dataset_name, size(collect(DISTINCT g1)) AS num_genes_all_species
    ORDER BY s1.species_scientific_name, c.{cell_type_col}, og.eggnog_dataset_name
    """

    with driver.session() as session:
        result_perct_all= session.run(query_perct_all)
        df_perct_all = result_perct_all.to_df()

    return df_perct_all


In [57]:

get_all_enriched('cell_type_name').to_csv("cell_type_enriched_genes_per_ct_all_datasets_cell_type_name.csv", index=False)
get_all_enriched('broad_type').to_csv("cell_type_enriched_genes_per_ct_all_datasets_broad_type.csv", index=False)
get_all_enriched('broad_type_2').to_csv("cell_type_enriched_genes_per_ct_all_datasets_broad_type_2.csv", index=False)
get_all_enriched('broad_type_3').to_csv("cell_type_enriched_genes_per_ct_all_datasets_broad_type_3.csv", index=False)

# sum per species per ct minus intersection is the union

In [58]:
df_ct_all = pd.read_csv("cell_type_enriched_genes_same_og_all_to_all_all_datasets_cell_type_name.csv")
df_ct_all.head()

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,c2.cell_type_name,og.eggnog_dataset_name,num_gene_sp1,num_gene_sp2
0,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,12,12
1,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,mammalia,9,9
2,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,metazoa,13,14
3,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,primates,9,9
4,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,vertebrata,10,11


In [126]:
df_jaccard_all.head()

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,c2.cell_type_name,og.eggnog_dataset_name,num_gene_sp1,num_gene_sp2,num_genes_total_same_og,total_specific_genes_species_sp1,total_specific_genes_species_sp2
0,cjacchus,ggorilla,L2/3 intratelencephalic projecting glutamaterg...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,12,12,24,43.0,60.0
1,cjacchus,ggorilla,L4 intratelencephalic projecting glutamatergic...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,6,6,12,52.0,60.0
2,cjacchus,ggorilla,L5 extratelencephalic projecting glutamatergic...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,8,9,17,53.0,60.0
3,cjacchus,ggorilla,L5 intratelencephalic projecting glutamatergic...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,7,8,15,37.0,60.0
4,cjacchus,ggorilla,L6 intratelencephalic projecting glutamatergic...,L2/3 intratelencephalic projecting glutamaterg...,bilateria,10,11,21,53.0,60.0


In [130]:

for level in ['cell_type_name', 'broad_type', 'broad_type_2', 'broad_type_3']:

      df_ct_all = pd.read_csv(f"results/jaccard_genes/cell_type_enriched_genes_same_og_all_to_all_all_datasets_{level}.csv")
      df_perct_all = pd.read_csv(f"results/jaccard_genes/cell_type_enriched_genes_per_ct_all_datasets_{level}.csv")

      df_ct_all['num_genes_total_same_og'] = df_ct_all['num_gene_sp1'] + df_ct_all['num_gene_sp2']
      df_ct_all
      # represent genes from both species in shared OGs
      df_perct_all
      # represent total specific genes, irrespective of OGs, from a certain species
      df_lonely_genes = df_perct_all.loc[df_perct_all['og.eggnog_dataset_name'].isna(), :]
      df_lonely_genes

      df_lonely_genes.loc[df_lonely_genes['s1.species_scientific_name'] == 'ggorilla', :]
      df_lonely_genes.drop(columns=['og.eggnog_dataset_name'], inplace=True)
      df_lonely_genes
      df_perct_all = df_perct_all.loc[~df_perct_all['og.eggnog_dataset_name'].isna(), :]
      df_perct_all_new = pd.merge(df_perct_all, df_lonely_genes, on=['s1.species_scientific_name',  f"c.{level}"], how='left',  suffixes=('', '_lonely'))
      df_perct_all_new.num_genes_all_species_lonely.fillna(0, inplace=True)
      df_perct_all_new['total_specific_genes_species']= df_perct_all_new['num_genes_all_species'] + df_perct_all_new['num_genes_all_species_lonely']
      df_perct_all_new.drop(columns=['num_genes_all_species_lonely', 'num_genes_all_species'], inplace=True)
      df_perct_all_new.loc[df_perct_all_new['s1.species_scientific_name'] == 'ggorilla', :].dropna()
      df_jaccard_all = df_ct_all.merge(df_perct_all_new, on=["s1.species_scientific_name", f"c.{level}", "og.eggnog_dataset_name"]).rename(columns={"total_specific_genes_species": "total_specific_genes_species_sp1"}). \
            merge(df_perct_all_new, left_on=["s2.species_scientific_name", f"c2.{level}", "og.eggnog_dataset_name"], 
                  right_on=["s1.species_scientific_name", f"c.{level}", "og.eggnog_dataset_name"]).rename(columns={"total_specific_genes_species": "total_specific_genes_species_sp2"}).drop(['s1.species_scientific_name_y', f'c.{level}_y'], axis=1). \
                  rename(columns={"s1.species_scientific_name_x": "s1.species_scientific_name", f"c.{level}_x": f"c.{level}"})
      
      df_jaccard_all['union'] = df_jaccard_all['total_specific_genes_species_sp1'] + df_jaccard_all['total_specific_genes_species_sp2']  ## this is not really a jaccard index because intersection happens at OG level, not gene level
      # it is more like for genes in the same OG / total number of genes
      df_jaccard_all['jaccard'] = df_jaccard_all['num_genes_total_same_og'] / df_jaccard_all['union']
      df_jaccard_all.to_csv(f"results/jaccard_genes/jaccard_cell_type_enriched_all_to_all_all_datasets_{level}.csv", index=False)

In [ ]:
#OLD: not considering genes but OGs 


for level in ['cell_type_name', 'broad_type', 'broad_type_2', 'broad_type_3']:

      df_ct_all = pd.read_csv(f"cell_type_enriched_genes_same_og_all_to_all_all_datasets_{level}.csv")
      df_perct_all = pd.read_csv(f"cell_type_enriched_genes_per_ct_all_datasets_{level}.csv")

      df_jaccard_all = df_ct_all.merge(df_perct_all, on=["s1.species_scientific_name", f"c.{level}", "og.eggnog_dataset_name"]).rename(columns="total_specific_genes_species": "total_specific_genes_species_sp1"}). \
      merge(df_perct_all, left_on=["s2.species_scientific_name", f"c2.{level}", "og.eggnog_dataset_name"], 
            right_on=["s1.species_scientific_name", f"c.{level}", "og.eggnog_dataset_name"]).rename(columns={"total_specific_genes_species": "total_specific_genes_species_sp2"}).drop(['s1.species_scientific_name_y', f'c.{level}_y'], axis=1). \
            rename(columns={"s1.species_scientific_name_x": "s1.species_scientific_name", f"c.{level}_x": f"c.{level}"})
      
      df_jaccard_all['union'] = df_jaccard_all['og_all_sp1'] + df_jaccard_all['og_all_sp2'] - df_jaccard_all['intersection']
      df_jaccard_all['jaccard'] = df_jaccard_all['intersection'] / df_jaccard_all['union']
      df_jaccard_all.to_csv(f"jaccard_cell_type_enriched_all_to_all_all_datasets_{level}.csv", index=False)



      

In [ ]:
## moved all result csvs to /Users/ysong/SOFTWARE/GeneSpectraKG/results/jaccard

In [ ]:
cell_type_col="cell_type_name"

query_per_species_enriched = f"""MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup),
    (g)-[enr:GeneEnrichedInCellType]->(c:CellType)
    RETURN s1.species_scientific_name, c.{cell_type_col}, og.id, og.eggnog_dataset_name, g.external_gene_name, g.id, g.ensembl_peptide_id, enr.specificity_score
    ORDER BY s1.species_scientific_name, c.{cell_type_col}"""

with driver.session() as session:
    result_query_per_species_enriched = session.run(query_per_species_enriched)
    df_per_species_enriched = result_query_per_species_enriched.to_df()

df_per_species_enriched.head()

,s1.species_scientific_name,c.cell_type_name,og.id,og.eggnog_dataset_name,g.external_gene_name,g.id,g.ensembl_peptide_id,enr.specificity_score
0,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,9G5A4,vertebrata,NELL1,ENSCJAG00000034303,ENSCJAP00000068199,4.338156833102107
1,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,HSY91,metazoa,NELL1,ENSCJAG00000034303,ENSCJAP00000068199,4.338156833102107
2,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,8YXGD,mammalia,NELL1,ENSCJAG00000034303,ENSCJAP00000068199,4.338156833102107
3,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,H4QFH,bilateria,NELL1,ENSCJAG00000034303,ENSCJAP00000068199,4.338156833102107
4,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,4ZJ0E,primates,NELL1,ENSCJAG00000034303,ENSCJAP00000068199,4.338156833102107


In [ ]:
df_per_species_enriched.to_csv("per_species_enriched.csv", index=False)

In [ ]:
cell_type_col="cell_type_name"

query_per_species_enhanced = f"""MATCH (s1:Species)<-[:GeneFromSpecies]-(g:Gene)-[:GeneInOrthologousGroup]->(og:OrthologousGroup),
    (g)-[enh:GeneEnhancedInCellType]->(c:CellType)
    RETURN s1.species_scientific_name, c.{cell_type_col}, og.id, og.eggnog_dataset_name, g.external_gene_name, g.id, g.ensembl_peptide_id, enh.specificity_score
    ORDER BY s1.species_scientific_name, c.{cell_type_col}"""

with driver.session() as session:
    result_query_per_species_enhanced = session.run(query_per_species_enhanced)
    df_per_species_enhanced = result_query_per_species_enhanced.to_df()

df_per_species_enhanced.head()

,s1.species_scientific_name,c.cell_type_name,og.id,og.eggnog_dataset_name,g.external_gene_name,g.id,g.ensembl_peptide_id,enh.specificity_score
0,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,HV1DW,metazoa,SIDT1,ENSCJAG00000020266,ENSCJAP00000037695,4.783335208892822
1,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,8Z4MV,mammalia,SIDT1,ENSCJAG00000020266,ENSCJAP00000037695,4.783335208892822
2,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,H4DDD,bilateria,SIDT1,ENSCJAG00000020266,ENSCJAP00000037695,4.783335208892822
3,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,502DJ,primates,SIDT1,ENSCJAG00000020266,ENSCJAP00000037695,4.783335208892822
4,cjacchus,L2/3 intratelencephalic projecting glutamaterg...,9G8PQ,vertebrata,SIDT1,ENSCJAG00000020266,ENSCJAP00000037695,4.783335208892822


In [ ]:
df_per_species_enhanced.to_csv("per_species_enhanced.csv", index=False)

In [ ]:
df_per_species_enhanced.shape

(70977, 8)

In [ ]:
df_jaccard = df_ct.merge(df_union, on=['s1.species_scientific_name', 's2.species_scientific_name', 'c.cell_type_name'])

In [ ]:
df_jaccard['union'] = df_jaccard['og_1'] + df_jaccard['og_2'] - df_jaccard['intersection']

In [ ]:
df_jaccard['jaccard'] =  df_jaccard['intersection'] / df_jaccard['union']

In [ ]:
df_jaccard.sort_values(by=['jaccard'], ascending=False)

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,intersection,og_1,og_2,union,jaccard
148,ggorilla,ptroglodytes,oligodendrocyte,183,324,327,468,0.391026
358,ptroglodytes,ggorilla,oligodendrocyte,183,327,324,468,0.391026
232,hsapiens,ptroglodytes,oligodendrocyte,197,382,327,512,0.384766
379,ptroglodytes,hsapiens,oligodendrocyte,197,327,382,512,0.384766
357,ptroglodytes,ggorilla,astrocyte of the cerebral cortex,231,416,418,603,0.383085
...,...,...,...,...,...,...,...,...
335,mmulatta,ptroglodytes,L6 intratelencephalic projecting glutamatergic...,9,54,65,110,0.081818
230,hsapiens,mmulatta,L6 intratelencephalic projecting glutamatergic...,9,68,54,113,0.079646
314,mmulatta,hsapiens,L6 intratelencephalic projecting glutamatergic...,9,54,68,113,0.079646
303,mmulatta,hsapiens,L5 extratelencephalic projecting glutamatergic...,8,58,67,117,0.068376


In [ ]:
df_jaccard.sort_values(by=['jaccard'], ascending=False).to_csv("cell_type_og_jaccard_1TPM_metazoa.csv")

In [ ]:
query_ct="""MATCH (n:CellType)
RETURN n.cell_type_name"""

with driver.session() as session:
    result_ct = session.run(query_ct)
    df_ct = result_ct.to_df()

print(df_ct)

                                     n.cell_type_name
0                    astrocyte of the cerebral cortex
1                                     oligodendrocyte
2                        vascular leptomeningeal cell
3                                     microglial cell
4                      oligodendrocyte precursor cell
5                    cerebral cortex endothelial cell
6       near-projecting glutamatergic cortical neuron
7   corticothalamic-projecting glutamatergic corti...
8                   L6b glutamatergic cortical neuron
9   L5 extratelencephalic projecting glutamatergic...
10  caudal ganglionic eminence derived GABAergic c...
11                 vip GABAergic cortical interneuron
12                sncg GABAergic cortical interneuron
13               lamp5 GABAergic cortical interneuron
14                 sst GABAergic cortical interneuron
15               pvalb GABAergic cortical interneuron
16    chandelier pvalb GABAergic cortical interneuron
17  L5 intratelencephalic pr

In [ ]:
cell_type_now=df_ct.iloc[0,0]
cell_type_now

'astrocyte of the cerebral cortex'

In [ ]:
query="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g3:Gene)-[:GeneFromSpecies]->(s1)
WITH s1,  COUNT(DISTINCT g3) AS total1, ct
RETURN s1.species_scientific_name, total1"""

with driver.session() as session:
    result = session.run(query)
    df = result.to_df()

print(df)

  s1.species_scientific_name  total1
0                   cjacchus     221
1                   mmulatta     295
2                   hsapiens     320
3                   ggorilla     312
4               ptroglodytes     303


In [ ]:
query_all_enrich="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType)<-[:GeneEnrichedInCellType]-(g3:Gene)-[:GeneFromSpecies]->(s1)
RETURN s1.species_scientific_name, ct.cell_type_name, g3.external_gene_name"""

with driver.session() as session:
    result_all_enrich = session.run(query_all_enrich)
    df_all_enrich = result_all_enrich.to_df()

df_all_enrich.head()
df_all_enrich.to_csv("all_enriched_genes.csv")

In [ ]:
query2="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g1:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
    (g1:Gene)-[:GeneFromSpecies]->(s1),
    (s2:Species)<-[:CellTypeFromSpecies]-(ct)<-[:GeneEnrichedInCellType]-(g2:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
    (g2:Gene)-[:GeneFromSpecies]->(s2)
WHERE s1 <> s2
WITH s1, s2, COUNT(DISTINCT g1) AS genes1, COUNT(DISTINCT g2) AS genes2, ct
RETURN s1.species_scientific_name, s2.species_scientific_name, genes1, genes2"""

with driver.session() as session:
    result2 = session.run(query2)
    df2 = result2.to_df()

print(df2)

   s1.species_scientific_name s2.species_scientific_name  genes1  genes2
0                    cjacchus               ptroglodytes      93      94
1                    cjacchus                   ggorilla      93      94
2                    cjacchus                   mmulatta      94      96
3                    cjacchus                   hsapiens      89      92
4                    mmulatta                   ggorilla     135     134
5                    mmulatta                   hsapiens     147     147
6                    mmulatta               ptroglodytes     141     139
7                    mmulatta                   cjacchus      96      94
8                    hsapiens               ptroglodytes     167     166
9                    hsapiens                   ggorilla     148     147
10                   hsapiens                   cjacchus      92      89
11                   hsapiens                   mmulatta     147     147
12                   ggorilla                   hsa

In [ ]:
df_new = df.merge(df2)

In [ ]:
df_new

,s1.species_scientific_name,total1,s2.species_scientific_name,genes1,genes2
0,cjacchus,221,ptroglodytes,93,94
1,cjacchus,221,ggorilla,93,94
2,cjacchus,221,mmulatta,94,96
3,cjacchus,221,hsapiens,89,92
4,mmulatta,295,ggorilla,135,134
5,mmulatta,295,hsapiens,147,147
6,mmulatta,295,ptroglodytes,141,139
7,mmulatta,295,cjacchus,96,94
8,hsapiens,320,ptroglodytes,167,166
9,hsapiens,320,ggorilla,148,147


In [ ]:
df_new.columns = ['sp1', 'sp1_total_enriched', 'sp2', 'num_enriched_share_og_sp1', 'num_enriched_share_og_sp2']

In [ ]:
df2

,s1.species_scientific_name,s2.species_scientific_name,genes1,genes2
0,cjacchus,ptroglodytes,93,94
1,cjacchus,ggorilla,93,94
2,cjacchus,mmulatta,94,96
3,cjacchus,hsapiens,89,92
4,mmulatta,ggorilla,135,134
5,mmulatta,hsapiens,147,147
6,mmulatta,ptroglodytes,141,139
7,mmulatta,cjacchus,96,94
8,hsapiens,ptroglodytes,167,166
9,hsapiens,ggorilla,148,147


In [ ]:
df_all = df_new.merge(df, left_on='sp2', right_on='s1.species_scientific_name').drop('s1.species_scientific_name', axis=1)

In [ ]:
df_all = df_all.rename(columns = {'total1':'sp2_total_enriched'})

In [ ]:
df_all

,sp1,sp1_total_enriched,sp2,num_enriched_share_og_sp1,num_enriched_share_og_sp2,sp2_total_enriched
0,cjacchus,221,ptroglodytes,93,94,303
1,mmulatta,295,ptroglodytes,141,139,303
2,hsapiens,320,ptroglodytes,167,166,303
3,ggorilla,312,ptroglodytes,162,165,303
4,cjacchus,221,ggorilla,93,94,312
5,mmulatta,295,ggorilla,135,134,312
6,hsapiens,320,ggorilla,148,147,312
7,ptroglodytes,303,ggorilla,165,162,312
8,cjacchus,221,mmulatta,94,96,295
9,hsapiens,320,mmulatta,147,147,295


In [ ]:
df_all['sp1_pct'] = df_all['num_enriched_share_og_sp1'] / df_all['sp1_total_enriched']
df_all['sp2_pct'] = df_all['num_enriched_share_og_sp2'] / df_all['sp2_total_enriched']

In [ ]:
df_all['hmean_pct'] = df_all.apply(lambda x: harmonic_mean([x['sp1_pct'], x['sp2_pct']]), axis=1)

In [ ]:
df_all

,sp1,sp1_total_enriched,sp2,num_enriched_share_og_sp1,num_enriched_share_og_sp2,sp2_total_enriched,sp1_pct,sp2_pct,hmean_pct
0,cjacchus,221,ptroglodytes,93,94,303,0.420814,0.310231,0.357159
1,mmulatta,295,ptroglodytes,141,139,303,0.477966,0.458746,0.468159
2,hsapiens,320,ptroglodytes,167,166,303,0.521875,0.547855,0.534549
3,ggorilla,312,ptroglodytes,162,165,303,0.519231,0.544554,0.531591
4,cjacchus,221,ggorilla,93,94,312,0.420814,0.301282,0.351155
5,mmulatta,295,ggorilla,135,134,312,0.457627,0.429487,0.443111
6,hsapiens,320,ggorilla,148,147,312,0.462500,0.471154,0.466787
7,ptroglodytes,303,ggorilla,165,162,312,0.544554,0.519231,0.531591
8,cjacchus,221,mmulatta,94,96,295,0.425339,0.325424,0.368733
9,hsapiens,320,mmulatta,147,147,295,0.459375,0.498305,0.478049


In [ ]:
df_all.sort_values('hmean_pct', ascending=False)[['sp1', 'sp2', 'hmean_pct']].drop_duplicates('hmean_pct').reset_index(drop=True)

,sp1,sp2,hmean_pct
0,hsapiens,ptroglodytes,0.534549
1,ggorilla,ptroglodytes,0.531591
2,hsapiens,mmulatta,0.478049
3,mmulatta,ptroglodytes,0.468159
4,ggorilla,hsapiens,0.466787
5,ggorilla,mmulatta,0.443111
6,mmulatta,cjacchus,0.368733
7,cjacchus,ptroglodytes,0.357159
8,cjacchus,ggorilla,0.351155
9,cjacchus,hsapiens,0.335491


In [ ]:
df_all['cell_type'] = cell_type_now

In [ ]:
df_all

,sp1,sp1_total_enriched,sp2,num_enriched_share_og_sp1,num_enriched_share_og_sp2,sp2_total_enriched,sp1_pct,sp2_pct,hmean_pct,cell_type
0,cjacchus,221,ptroglodytes,93,94,303,0.420814,0.310231,0.357159,astrocyte of the cerebral cortex
1,mmulatta,295,ptroglodytes,141,139,303,0.477966,0.458746,0.468159,astrocyte of the cerebral cortex
2,hsapiens,320,ptroglodytes,167,166,303,0.521875,0.547855,0.534549,astrocyte of the cerebral cortex
3,ggorilla,312,ptroglodytes,162,165,303,0.519231,0.544554,0.531591,astrocyte of the cerebral cortex
4,cjacchus,221,ggorilla,93,94,312,0.420814,0.301282,0.351155,astrocyte of the cerebral cortex
5,mmulatta,295,ggorilla,135,134,312,0.457627,0.429487,0.443111,astrocyte of the cerebral cortex
6,hsapiens,320,ggorilla,148,147,312,0.462500,0.471154,0.466787,astrocyte of the cerebral cortex
7,ptroglodytes,303,ggorilla,165,162,312,0.544554,0.519231,0.531591,astrocyte of the cerebral cortex
8,cjacchus,221,mmulatta,94,96,295,0.425339,0.325424,0.368733,astrocyte of the cerebral cortex
9,hsapiens,320,mmulatta,147,147,295,0.459375,0.498305,0.478049,astrocyte of the cerebral cortex


In [ ]:
data_final = pd.DataFrame()

for cell_type_now in df_ct.iloc[:, 0]:
    print(cell_type_now)

    # get all cell type senriched genes
    query="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result = session.run(query)
        df = result.to_df()

    # get cell type enriched genes in same OG
    query2="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType]-(g1:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
        (g1:Gene)-[:GeneFromSpecies]->(s1),
        (s2:Species)<-[:CellTypeFromSpecies]-(ct)<-[:GeneEnrichedInCellType]-(g2:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
        (g2:Gene)-[:GeneFromSpecies]->(s2)
    WHERE s1 <> s2
    WITH s1, s2, COUNT(DISTINCT g1) AS genes1, COUNT(DISTINCT g2) AS genes2, ct
    RETURN s1.species_scientific_name, s2.species_scientific_name, genes1, genes2"""

    with driver.session() as session:
        result2 = session.run(query2)
        df2 = result2.to_df()

    # calculate pct and hmean
    df_new = df.merge(df2)
    df_new.columns = ['sp1', 'sp1_total_enriched', 'sp2', 'num_enriched_share_og_sp1', 'num_enriched_share_og_sp2']
    df_all = df_new.merge(df, left_on='sp2', right_on='s1.species_scientific_name').drop('s1.species_scientific_name', axis=1)
    df_all = df_all.rename(columns = {'total1':'sp2_total_enriched'})
    df_all['sp1_pct'] = df_all['num_enriched_share_og_sp1'] / df_all['sp1_total_enriched']
    df_all['sp2_pct'] = df_all['num_enriched_share_og_sp2'] / df_all['sp2_total_enriched']
    df_all['hmean_pct'] = df_all.apply(lambda x: harmonic_mean([x['sp1_pct'], x['sp2_pct']]), axis=1)
    df_all['cell_type'] = cell_type_now
    df_all['specifcity_category'] = 'gene enriched in cell type'

    if data_final.shape[0] == 0:
        data_final = df_all
    else:
        data_final = pd.concat([data_final, df_all]).reset_index(drop=True)


    
    


    

astrocyte of the cerebral cortex
oligodendrocyte
vascular leptomeningeal cell
microglial cell
oligodendrocyte precursor cell
cerebral cortex endothelial cell
near-projecting glutamatergic cortical neuron
corticothalamic-projecting glutamatergic cortical neuron
L6b glutamatergic cortical neuron
L5 extratelencephalic projecting glutamatergic cortical neuron
caudal ganglionic eminence derived GABAergic cortical interneuron
vip GABAergic cortical interneuron
sncg GABAergic cortical interneuron
lamp5 GABAergic cortical interneuron
sst GABAergic cortical interneuron
pvalb GABAergic cortical interneuron
chandelier pvalb GABAergic cortical interneuron
L5 intratelencephalic projecting glutamatergic neuron
L4 intratelencephalic projecting glutamatergic neuron
L2/3 intratelencephalic projecting glutamatergic neuron
L6 intratelencephalic projecting glutamatergic neuron


In [ ]:
data_final.head(10)

,sp1,sp1_total_enriched,sp2,num_enriched_share_og_sp1,num_enriched_share_og_sp2,sp2_total_enriched,sp1_pct,sp2_pct,hmean_pct,cell_type,specifcity_category
0,ptroglodytes,458,ggorilla,238,233,441,0.519651,0.528345,0.523962,astrocyte of the cerebral cortex,gene enriched in cell type
1,cjacchus,296,ggorilla,122,124,441,0.412162,0.281179,0.334298,astrocyte of the cerebral cortex,gene enriched in cell type
2,mmulatta,409,ggorilla,183,183,441,0.447433,0.414966,0.430588,astrocyte of the cerebral cortex,gene enriched in cell type
3,hsapiens,456,ggorilla,218,216,441,0.478070,0.489796,0.483862,astrocyte of the cerebral cortex,gene enriched in cell type
4,ptroglodytes,458,cjacchus,129,126,296,0.281659,0.425676,0.339006,astrocyte of the cerebral cortex,gene enriched in cell type
5,ggorilla,441,cjacchus,124,122,296,0.281179,0.412162,0.334298,astrocyte of the cerebral cortex,gene enriched in cell type
6,mmulatta,409,cjacchus,131,127,296,0.320293,0.429054,0.366781,astrocyte of the cerebral cortex,gene enriched in cell type
7,hsapiens,456,cjacchus,121,118,296,0.265351,0.398649,0.318620,astrocyte of the cerebral cortex,gene enriched in cell type
8,ptroglodytes,458,mmulatta,195,195,409,0.425764,0.476773,0.449827,astrocyte of the cerebral cortex,gene enriched in cell type
9,cjacchus,296,mmulatta,127,131,409,0.429054,0.320293,0.366781,astrocyte of the cerebral cortex,gene enriched in cell type


In [ ]:
data_final.groupby(['cell_type']).apply(lambda x: x.sort_values(['hmean_pct'], ascending=False))[['sp1', 'sp2', 'hmean_pct', 'cell_type']].drop_duplicates('hmean_pct').reset_index(drop=True).to_csv("enriched_genes_hmean_pct.csv")


In [ ]:
data_final_enhanced = pd.DataFrame()

for cell_type_now in df_ct.iloc[:, 0]:
    print(cell_type_now)

    # get all cell type enriched genes
    query="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnhancedInCellType]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result = session.run(query)
        df = result.to_df()

    # get cell type enriched genes in same OG
    query2="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnhancedInCellType]-(g1:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
        (g1:Gene)-[:GeneFromSpecies]->(s1),
        (s2:Species)<-[:CellTypeFromSpecies]-(ct)<-[:GeneEnhancedInCellType]-(g2:Gene)-[:GeneInOrthologousGroup]->(gf:OrthologousGroup),
        (g2:Gene)-[:GeneFromSpecies]->(s2)
    WHERE s1 <> s2
    WITH s1, s2, COUNT(DISTINCT g1) AS genes1, COUNT(DISTINCT g2) AS genes2, ct
    RETURN s1.species_scientific_name, s2.species_scientific_name, genes1, genes2"""

    with driver.session() as session:
        result2 = session.run(query2)
        df2 = result2.to_df()

    # calculate pct and hmean
    df_new = df.merge(df2)
    df_new.columns = ['sp1', 'sp1_total_enriched', 'sp2', 'num_enriched_share_og_sp1', 'num_enriched_share_og_sp2']
    df_all = df_new.merge(df, left_on='sp2', right_on='s1.species_scientific_name').drop('s1.species_scientific_name', axis=1)
    df_all = df_all.rename(columns = {'total1':'sp2_total_enriched'})
    df_all['sp1_pct'] = df_all['num_enriched_share_og_sp1'] / df_all['sp1_total_enriched']
    df_all['sp2_pct'] = df_all['num_enriched_share_og_sp2'] / df_all['sp2_total_enriched']
    df_all['hmean_pct'] = df_all.apply(lambda x: harmonic_mean([x['sp1_pct'], x['sp2_pct']]), axis=1)
    df_all['cell_type'] = cell_type_now
    df_all['specifcity_category'] = 'gene enhanced in cell type'

    if data_final.shape[0] == 0:
        data_final_enhanced = df_all
    else:
        data_final_enhanced = pd.concat([data_final_enhanced, df_all]).reset_index(drop=True)

astrocyte of the cerebral cortex
oligodendrocyte
vascular leptomeningeal cell
microglial cell
oligodendrocyte precursor cell
cerebral cortex endothelial cell
near-projecting glutamatergic cortical neuron
corticothalamic-projecting glutamatergic cortical neuron
L6b glutamatergic cortical neuron
L5 extratelencephalic projecting glutamatergic cortical neuron
caudal ganglionic eminence derived GABAergic cortical interneuron
vip GABAergic cortical interneuron
sncg GABAergic cortical interneuron
lamp5 GABAergic cortical interneuron
sst GABAergic cortical interneuron
pvalb GABAergic cortical interneuron
chandelier pvalb GABAergic cortical interneuron
L5 intratelencephalic projecting glutamatergic neuron
L4 intratelencephalic projecting glutamatergic neuron
L2/3 intratelencephalic projecting glutamatergic neuron
L6 intratelencephalic projecting glutamatergic neuron


In [ ]:
pd.concat([data_final, data_final_enhanced]).reset_index(drop=True).to_csv("sum_enhanced_enriched_1TPM.csv")

In [28]:
df_ct

,s1.species_scientific_name,s2.species_scientific_name,c.cell_type_name,intersection
0,cjacchus,ggorilla,vip GABAergic cortical interneuron,13
1,cjacchus,ggorilla,sncg GABAergic cortical interneuron,9
2,cjacchus,ggorilla,microglial cell,206
3,cjacchus,ggorilla,vascular leptomeningeal cell,203
4,cjacchus,ggorilla,oligodendrocyte precursor cell,63
...,...,...,...,...
415,ptroglodytes,mmulatta,caudal ganglionic eminence derived GABAergic c...,23
416,ptroglodytes,mmulatta,vip GABAergic cortical interneuron,29
417,ptroglodytes,mmulatta,sncg GABAergic cortical interneuron,23
418,ptroglodytes,mmulatta,sst GABAergic cortical interneuron,17


In [33]:
data_final_ct = pd.DataFrame()

for cell_type_now in df_ct.iloc[:, 2]:
    print(cell_type_now)


    query1="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnhancedInCellType {specificity_category: 'cell type enhanced'}]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result1 = session.run(query1)
        df1 = result1.to_df()

    df1['specificity_category'] = 'cell type enhanced'
    df1['cell_type'] = cell_type_now


    query2="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType {specificity_category: 'cell type enriched'}]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result2 = session.run(query2)
        df2 = result2.to_df()

    df2['specificity_category'] = 'cell type enriched'
    df2['cell_type'] = cell_type_now

    query3="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnhancedInCellType {specificity_category: 'group enhanced'}]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result3 = session.run(query3)
        df3 = result3.to_df()

    df3['specificity_category'] = 'group enhanced'
    df3['cell_type'] = cell_type_now


    query4="""MATCH (s1:Species)<-[:CellTypeFromSpecies]-(ct:CellType {cell_type_name: '""" + cell_type_now + """'})<-[:GeneEnrichedInCellType {specificity_category: 'group enriched'}]-(g3:Gene)-[:GeneFromSpecies]->(s1)
    WITH s1,  COUNT(DISTINCT g3) AS total1, ct
    RETURN s1.species_scientific_name, total1"""

    with driver.session() as session:
        result4 = session.run(query4)
        df4 = result4.to_df()

    df4['specificity_category'] = 'group enriched'
    df4['cell_type'] = cell_type_now


    df_all=pd.concat([df1, df2, df3, df4]).reset_index(drop=True)

    if data_final_ct.shape[0] == 0:
        data_final_ct = df_all
    else:
        data_final_ct= pd.concat([data_final_ct, df_all]).reset_index(drop=True)
   

vip GABAergic cortical interneuron
sncg GABAergic cortical interneuron
microglial cell
vascular leptomeningeal cell
oligodendrocyte precursor cell
cerebral cortex endothelial cell
oligodendrocyte
astrocyte of the cerebral cortex
lamp5 GABAergic cortical interneuron
caudal ganglionic eminence derived GABAergic cortical interneuron
L4 intratelencephalic projecting glutamatergic neuron
near-projecting glutamatergic cortical neuron
corticothalamic-projecting glutamatergic cortical neuron
L6b glutamatergic cortical neuron
L5 intratelencephalic projecting glutamatergic neuron
L6 intratelencephalic projecting glutamatergic neuron
L5 extratelencephalic projecting glutamatergic cortical neuron
L2/3 intratelencephalic projecting glutamatergic neuron
chandelier pvalb GABAergic cortical interneuron
pvalb GABAergic cortical interneuron
sst GABAergic cortical interneuron
microglial cell
oligodendrocyte precursor cell
vascular leptomeningeal cell
cerebral cortex endothelial cell
sncg GABAergic cortic

In [38]:
data_final_ct.drop_duplicates().to_csv("sum_enhanced_enriched_1TPM_by_category.csv")